# Import Libraries

In [1]:
#Libraires import
import quandl
import pandas as pd

# Early Definitions

In [2]:
#Quandl API key - Created a Free Account
quandl.ApiConfig.api_key = ""

In [3]:
#Import the DataFrame with Country Names and Country ISO Codes (for Download)
countries_df = pd.read_csv('Country_ISO_code.csv')

# Obtaining the Data

In [4]:
#This is the code for the 1st Country to be Downloaded
##Note: A DataFrame will be created with the data for the 1st country and then ...
## ... the data for the next countries will be added to this DataFrame in a loop via merger (see next cell)

#Define the Country Name and the Country ISO Code 
country_name = countries_df.iloc[0]['Country']
country_code = countries_df.iloc[0]['ISO3']

#Codes to Download the Debt level by GDP and Outstanding Debt Value using the Country ISO Code
country_debt_to_gdp = quandl.get("ODA/"+country_code+"_GGXWDG_NGDP")
country_gdp = quandl.get("ODA/"+country_code+"_NGDPD")

#Reset Index for Merging based on "Date" column
country_debt_to_gdp = country_debt_to_gdp.reset_index()
country_gdp = country_gdp.reset_index()

#Change the Name of the column with the Data from Value to the Country's Name
country_debt_to_gdp = country_debt_to_gdp.rename(columns={'Value': country_name})
country_gdp = country_gdp.rename(columns={'Value': country_name})

In [5]:
#Loop to Obtain the data for each Country In the Countries DataFrame
##Same code as above with difference in the last 2 lines
for index, row in countries_df[1:].iterrows():
    try:
        country_name = row['Country']
        country_code = row['ISO3']

        country_debt_to_gdp_next = quandl.get("ODA/"+country_code+"_GGXWDG_NGDP")
        country_gdp_next = quandl.get("ODA/"+country_code+"_NGDPD")

        country_debt_to_gdp_next = country_debt_to_gdp_next.reset_index()
        country_gdp_next = country_gdp_next.reset_index()

        country_debt_to_gdp_next = country_debt_to_gdp_next.rename(columns={'Value': country_name})
        country_gdp_next = country_gdp_next.rename(columns={'Value': country_name})

        #Merge this with the main DataFrame
        country_debt_to_gdp = country_debt_to_gdp.merge(country_debt_to_gdp_next, on='Date', how='outer')
        country_gdp = country_gdp.merge(country_gdp_next, on='Date', how='outer')
    except:
        continue

In [6]:
#Sort DataFrames by Date
country_debt_to_gdp = country_debt_to_gdp.sort_values(by=['Date'])
country_gdp = country_gdp.sort_values(by=['Date'])

In [7]:
#Delete prospective rows
country_debt_to_gdp = country_debt_to_gdp.iloc[:40]
country_gdp = country_gdp.iloc[:40]

In [8]:
#DataFrame for the Latest Value for GDP in USD Billions
country_gdp_latest = pd.DataFrame(country_gdp.iloc[-1])
country_gdp_latest = country_gdp_latest[1:]
country_gdp_latest = country_gdp_latest.reset_index()
country_gdp_latest.columns = ['Country','GDP']

#DataFrame for the Latest Value for Debt to GDP
country_debt_to_gdp_latest = pd.DataFrame(country_debt_to_gdp.iloc[-1])
country_debt_to_gdp_latest = country_debt_to_gdp_latest[1:]
country_debt_to_gdp_latest = country_debt_to_gdp_latest.reset_index()
country_debt_to_gdp_latest.columns = ['Country','Debt to GDP']
country_debt_to_gdp_latest['Rank'] = country_debt_to_gdp_latest['Debt to GDP'].rank(ascending=False).dropna().astype('int32')

In [9]:
#DataFrame for the Latest Value for Debt in in USD Billions
country_debt_latest = country_gdp_latest.merge(country_debt_to_gdp_latest,on='Country', how='left')
country_debt_latest['Debt'] = country_debt_latest['GDP']*country_debt_latest['Debt to GDP']/100
country_debt_latest = country_debt_latest.drop(columns=['GDP','Debt to GDP', 'Rank'])

# Treemap of World Debt

In [10]:
#Libraries for Treemap
import plotly.express as px

In [11]:
fig = px.treemap(country_debt_latest, path=[px.Constant("World"), 'Country'], values='Debt')
fig.update_traces(root_color="lightgrey", textinfo='label+value+percent entry')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
#fig.show()
fig.write_html("Debt_by_Country_Treemap_Plotly.html")